In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('test_games.xlsx')
df.head(20)

,Country,1month,2month,3month,4month,Unnamed: 5,90D # of ratings,28D # of ratings
0,AL,4.80,4.20,5.00,5.00,NaN,355,100.0
1,AM,4.30,4.10,5.00,5.00,NaN,25,10.0
2,AR,4.80,4.82,4.75,4.78,NaN,15,5.0
3,AT,4.80,4.81,4.93,4.88,NaN,340,85.0
4,AU,4.50,4.54,4.75,4.39,NaN,5,5.0
5,BA,3.90,5.00,NaN,4.40,NaN,185,40.0
6,BD,4.30,NaN,5.00,5.00,NaN,290,85.0
7,BE,4.73,4.71,4.74,4.69,NaN,20,10.0
8,BG,4.73,4.75,4.89,4.81,NaN,65,30.0
9,BO,4.00,4.00,NaN,5.00,NaN,100,15.0


In [3]:
df = df.drop('Unnamed: 5', axis = 1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country           22 non-null     object 
 1   1month            21 non-null     float64
 2   2month            20 non-null     float64
 3   3month            19 non-null     float64
 4   4month            22 non-null     float64
 5   90D # of ratings  22 non-null     int64  
 6   28D # of ratings  21 non-null     float64
dtypes: float64(5), int64(1), object(1)
memory usage: 1.3+ KB


In [5]:
df = df.set_index('Country')

In [6]:
def empty(x):
    for i in range(0, 6):
        if x[i] != x[i]: print('Cтрана', x.name, 'показатель', x.index[i])

In [7]:
print('Нет данных по следующим странам/показателям')
print()
df.apply(empty, axis=1);

Нет данных по следующим странам/показателям

Cтрана BA показатель 3month
Cтрана BD показатель 2month
Cтрана BO показатель 3month
Cтрана BY показатель 1month
Cтрана BY показатель 28D # of ratings
Cтрана CR показатель 2month
Cтрана CZ показатель 3month


In [80]:
logs = pd.read_excel('test_ret.xlsx')

In [81]:
# предпоалагаем, что date - дата, date_time - дата и время в формате datetime
# для каждого пользователя находит первую дату регестрации 
reg = logs.query('event == "registration"').groupby(['userId']).agg({'date_time':'min'}).reset_index()

In [82]:
# добавляем с таблицу с логами дату регестарции
logs = logs.merge(reg, on ='userId')
logs = logs.rename({'date_time_x':'date_time', 'date_time_y' : 'reg_date_time'}, axis = 1)

In [83]:
# считаем время жизни
logs['lifetime'] = logs['date_time'] - logs['reg_date_time']
logs['lifetime'] = (logs['lifetime'] / np.timedelta64(1, 'D')).round().astype(int)

In [84]:
# добавим дату когорт
logs['regDate'] = logs['reg_date_time'].dt.strftime('%Y-%m-%d')

In [85]:
# посчитаем кол-во пользователей 
cohorts = logs.groupby(['regDate', 'lifetime']).agg({'userId':'nunique'}).reset_index()
cohorts = cohorts.rename({'userId' : 'n_users'}, axis = 1)

In [86]:
# посчитаем кол-во уникальных пользователей в когорте
initial_users = cohorts[cohorts['lifetime'] == 0][['regDate','n_users']]
initial_users = initial_users.rename({'n_users':'initial_users'}, axis = 1)

In [87]:
# объединим таблицы и посчитает retation rate
cohorts = cohorts.merge(initial_users, on = ['regDate'])
cohorts['retention_rate'] = cohorts['n_users'] / cohorts ['initial_users']

In [88]:
# выведим результаты в запрашиваемом виде 
itog = cohorts.pivot_table(index = 'regDate', columns = 'lifetime', values = 'retention_rate')
itog.index.name = None
itog.iloc[:, [1, 3, 6]].rename({1: 'ret1d', 3: 'ret3d', 7:'ret7d'}, axis = 1)

lifetime,ret1d,ret3d,ret7d
2021-12-01,1.0,NaN,NaN
2021-12-07,NaN,1.0,0.5
2021-12-14,NaN,NaN,NaN
